# Analyzing and Visualizing Retail Sales

1. Analyze and visualize retail sales by weekday, month, and year.
1. Determine seasonality.
1. Train-test split data for model ingestion.

# Imports and Setup

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import dotenv_values

from scipy import signal

import hvplot.pandas
pd.options.plotting.backend = 'holoviews'

In [3]:
# read .env and convert paths
config = dotenv_values()

for this_key in config.keys():
    if this_key.endswith('_dir'):
        globals()[this_key] = Path(config[this_key])


In [8]:
sales = pd.read_csv(processed_data_dir / 'sales_processed.csv', index_col=0, parse_dates=[1])

In [9]:
sales

,date,store_id,cat_id,sales
0,2011-01-29,TX_1,FOODS,3950.35
1,2011-01-30,TX_1,FOODS,3844.97
2,2011-01-31,TX_1,FOODS,2888.03
3,2011-02-01,TX_1,FOODS,3631.28
4,2011-02-02,TX_1,FOODS,3072.18
...,...,...,...,...
58225,2016-05-18,WI_3,HOUSEHOLD,2225.31
58226,2016-05-19,WI_3,HOUSEHOLD,2468.50
58227,2016-05-20,WI_3,HOUSEHOLD,3664.25
58228,2016-05-21,WI_3,HOUSEHOLD,3487.02


In [10]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58230 entries, 0 to 58229
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      58230 non-null  datetime64[ns]
 1   store_id  58230 non-null  object        
 2   cat_id    58230 non-null  object        
 3   sales     57402 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 2.2+ MB


# EDA

Plot line charts for each category, store, and category-store.

In [11]:
sales.hvplot(x='date', y='sales', groupby=['store_id', 'cat_id'], alpha=0.5).opts(width=900, height=500)

:DynamicMap   [store_id,cat_id]
   :Curve   [date]   (sales)

In [19]:
sales.hvplot(x='date', y='sales', by=['store_id', 'cat_id'], alpha=0.5, subplots=True, width=500).cols(2)

:NdLayout   [store_id,cat_id]
   :Curve   [date]   (sales)

Identify the time series with breaks in trend and remove the past data before the breaks from these time series. Save the output as sales_cleaned.csv in the data folder.

by inspection above, the following have breaks in trend:

CA-2 Foods 2015-06-01

WI-1 Foods 2012-10-25

WI-2 Foods 2012-06-01

In [28]:
# drop rows with dates before the break in trend
cat = 'FOODS'
store = 'CA_2'
dt = pd.Timestamp('2015-06-01')
sales.drop(sales[(sales['cat_id']==cat) & (sales['store_id']==store) & (sales['date']<dt)].index, inplace=True)

cat = 'FOODS'
store = 'WI_1'
dt = pd.Timestamp('2012-10-25')
sales.drop(sales[(sales['cat_id']==cat) & (sales['store_id']==store) & (sales['date']<dt)].index, inplace=True)

cat = 'FOODS'
store = 'WI_2'
dt = pd.Timestamp('2012-06-01')
sales.drop(sales[(sales['cat_id']==cat) & (sales['store_id']==store) & (sales['date']<dt)].index, inplace=True)

In [29]:
sales.shape

(55522, 4)

In [30]:
# check with same plot as above
sales.hvplot(x='date', y='sales', by=['store_id', 'cat_id'], alpha=0.5, subplots=True, width=500).cols(2)

:NdLayout   [store_id,cat_id]
   :Curve   [date]   (sales)

In [74]:
# save cleaned data
sales.to_csv(processed_data_dir / 'sales_cleaned.csv', index=False)

Choose a random store_id and cat_id to plot the box plots and line charts of the sales by weekday, day of month, and month separately.

In [44]:
# add weekday, day of month, and month to dataframe
sales['weekday'] = [x.weekday() for x in sales['date']]
sales['day_of_month'] = [x.day for x in sales['date']]
sales['month'] = [x.month for x in sales['date']]

In [45]:
sales

,date,store_id,cat_id,sales,weekday,day_of_month,month
0,2011-01-29,TX_1,FOODS,3950.35,5,29,1
1,2011-01-30,TX_1,FOODS,3844.97,6,30,1
2,2011-01-31,TX_1,FOODS,2888.03,0,31,1
3,2011-02-01,TX_1,FOODS,3631.28,1,1,2
4,2011-02-02,TX_1,FOODS,3072.18,2,2,2
...,...,...,...,...,...,...,...
58225,2016-05-18,WI_3,HOUSEHOLD,2225.31,2,18,5
58226,2016-05-19,WI_3,HOUSEHOLD,2468.50,3,19,5
58227,2016-05-20,WI_3,HOUSEHOLD,3664.25,4,20,5
58228,2016-05-21,WI_3,HOUSEHOLD,3487.02,5,21,5


In [52]:
sales.hvplot.box(y='sales', ylabel='Sales', by=['cat_id', 'weekday'], ylim=(0, 18000), width=900, height=600, groupby='store_id')

:DynamicMap   [store_id]
   :BoxWhisker   [cat_id,weekday]   (sales)

In [70]:
sales.hvplot.box(y='sales', ylabel='Sales', by=['cat_id', 'day_of_month'], ylim=(0, 18000), width=1000, height=600, groupby='store_id')

:DynamicMap   [store_id]
   :BoxWhisker   [cat_id,day_of_month]   (sales)

In [71]:
sales.hvplot.box(y='sales', ylabel='Sales', by=['cat_id', 'month'], ylim=(0, 18000), width=1000, height=600, groupby='store_id')

:DynamicMap   [store_id]
   :BoxWhisker   [cat_id,month]   (sales)

In [83]:
sales[(sales['store_id']=='WI_2') & (sales['cat_id']=='FOODS')].hvplot(x='date', y='sales', by='weekday', width=200, rot=90, subplots=True).cols(7)

:NdLayout   [weekday]
   :Curve   [date]   (sales)

Select a random time series and set up a periodogram using "scipy.signal.periodogram." Identify the main seasonality pattern(s).
Hints for Step 5

"scipy.signal.detrend" can be used to detrend the time series before setting up the periodogram. The frequency values from the periodogram can be inverted (1/freq) to get the time interval values.

In [86]:
tmp = sales[(sales['store_id']=='WI_2') & (sales['cat_id']=='FOODS')]
dt = tmp['date'].values
ts = tmp['sales'].values

In [93]:
ts_detrend = signal.detrend(ts)

ValueError: array must not contain infs or NaNs

In [90]:
dir(scipy)

['test']